<a href="https://colab.research.google.com/github/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemystified_seq2seq_and_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing Demystified | Seq2Seq and Attention
https://nlpdemystified.org<br>
https://github.com/nitinpunjabi/nlp-demystified<br><br>

Course module for this demo: https://www.nlpdemystified.org/course/seq2seq-and-attention

**IMPORTANT**<br>
Enable **GPU acceleration** by going to *Runtime > Change Runtime Type*. Keep in mind that, on certain tiers, you're not guaranteed GPU access depending on usage history and current load.
<br><br>
Also, if you're running this in the cloud rather than a local Jupyter server on your machine, then the notebook will *timeout* after a period of inactivity.
<br><br>
Refer to this link on how to run Colab notebooks locally on your machine to avoid this issue:<br>
https://research.google.com/colaboratory/local-runtimes.html

In [ ]:
import io
import json
import numpy as np
import pandas as pd
import random
import re
import tensorflow as tf
import unicodedata

from google.colab import files
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Recurrence-based Seq2Seq Neural Machine Translation **WITHOUT** Attention

Our first model will use just two LSTMs (one encoder and one decoder) to translate Hungarian to English. We're going with Hungarian here because it's a particularly challenging language to tackle with few resources.
<br><br>
The dataset we'll use comes from **Tatoeba**, a collection of sentence translations in a variety of languages sourced from volunteers:
<br>
https://tatoeba.org/en

The **NLP Demystified** repo has a lightly processed copy that's already been split into train/validation/test sets and pre-shuffled.

In [ ]:
# Download the training set.
!wget https://raw.githubusercontent.com/nitinpunjabi/nlp-demystified/main/datasets/hun_eng_pairs/hun_eng_pairs_train.txt

In [ ]:
with open('hun_eng_pairs_train.txt') as file:
  train = [line.rstrip() for line in file]

Each entry consists of a Hungarian sentence followed by its English translation, separated by a **\<sep\>** delimiter

In [ ]:
train[:3]

This is a relatively tiny dataset for neural machine translation, but we'll see how far we can get.

In [ ]:
len(train)

In [ ]:
# Separate the input (Hungarian) and target (English) sentences into separate lists.
SEPARATOR = '<sep>'
train_input, train_target = map(list, zip(*[pair.split(SEPARATOR) for pair in train]))

In [ ]:
print(train_input[:3])
print(train_target[:3])

Since we're dealing with a source language that uses accented characters, it's important to apply *Unicode normalization* 
<br><br>
In the example below, two different sets of Unicode yield the same character visually. The first Unicode is for an accented 'a', while the second Unicode combines an 'a' with an accent mark.

In [ ]:
print("\u00E1", "\u0061\u0301")

Though these characters look the same to us reading them, they'll be treated differently by a model. So to avoid this, the following function normalizes any accented characters into the same set of Unicode, and then replaces them with their ASCII equivalents.<br>
https://docs.python.org/3/library/unicodedata.html
<br><br>
Here's an informative article on the importance of Unicode normalization and how to go about it (including what _NFD_ means):<br>
https://towardsdatascience.com/what-on-earth-is-unicode-normalization-56c005c55ad0

In [ ]:
# Unicode normalization
def normalize_unicode(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

We're building a **word**-based translation model, but we still want to keep punctuation and treat them as separate tokens, so we'll insert a space between any relevant punctuation and the characters around them. This way, our tokenizer (which won't filter out punctuation) will output punctuation marks as separate tokens.

This function does that and unicode normalization.

In [ ]:
def preprocess_sentence(s):
  s = normalize_unicode(s)
  s = re.sub(r"([?.!,¿])", r" \1 ", s)
  s = re.sub(r'[" "]+', " ", s)
  s = s.strip()
  return s

In [ ]:
# Preprocess both the source and target sentences.
train_preprocessed_input = [preprocess_sentence(s) for s in train_input]
train_preprocessed_target = [preprocess_sentence(s) for s in train_target]

After preprocessing, the unicode should be normalized and there should be spaces on either side of any punctuation.

In [ ]:
train_preprocessed_input[:3]

Like the language model we built in the [previous demo](https://github.com/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemystified_recurrent_neural_networks.ipynb), we'll use **Teacher Forcing** with our translation model (specifically, the decoder). This begins by placing a  start-of-sentence tag (_\<sos\>_) and end-of-sentence tag (_\<eos\>_) at the beginning and end of each target sentence, respectively.

In [ ]:
def tag_target_sentences(sentences):
  tagged_sentences = map(lambda s: (' ').join(['<sos>', s, '<eos>']), sentences)
  return list(tagged_sentences)

In [ ]:
train_tagged_preprocessed_target = tag_target_sentences(train_preprocessed_target)

In [ ]:
train_tagged_preprocessed_target[:3]

Next, we'll tokenize our input and target sentences, taking care to keep relevant punctuation.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
<br><br>
Note that we're also including an out-of-vocabulary token (_\<unk\>_) in the tokenizer initialization. At inference time, if the tokenizer encounters a word it didn't see during the initial fit on the training data, that word will be replaced with _\<unk\>_ and the translation system will need to cope with it.

In [ ]:
# Tokenizer for the Hungarian input sentences. Note how we're not filtering punctuation.
source_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<unk>', filters='"#$%&()*+-/:;=@[\\]^_`{|}~\t\n')
source_tokenizer.fit_on_texts(train_preprocessed_input)
source_tokenizer.get_config()

In [ ]:
source_vocab_size = len(source_tokenizer.word_index) + 1
print(source_vocab_size)

In [ ]:
# Tokenizer for the English target sentences.
target_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<unk>', filters='"#$%&()*+-/:;=@[\\]^_`{|}~\t\n')
target_tokenizer.fit_on_texts(train_tagged_preprocessed_target)
target_tokenizer.get_config()

In [ ]:
target_vocab_size = len(target_tokenizer.word_index) + 1
print(target_vocab_size)

Next, we'll vectorize the input and target sentences just like we did in the last few demos.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#texts_to_sequences

In [ ]:
train_encoder_inputs = source_tokenizer.texts_to_sequences(train_preprocessed_input)

In [ ]:
print(train_encoder_inputs[:3])
print(source_tokenizer.sequences_to_texts(train_encoder_inputs[:3]))

For teacher forcing, we'll create two copies of each vectorized **target** sentence, with the second copy shifted over by one.

The function below takes a collection of sentences, vectorizes them, then returns two copies of each. The first will include every token except the last, the second will include every token except the first.

In [ ]:
def generate_decoder_inputs_targets(sentences, tokenizer):
  seqs = tokenizer.texts_to_sequences(sentences)
  decoder_inputs = [s[:-1] for s in seqs] # Drop the last token in the sentence.
  decoder_targets = [s[1:] for s in seqs] # Drop the first token in the sentence.

  return decoder_inputs, decoder_targets

In [ ]:
train_decoder_inputs, train_decoder_targets = generate_decoder_inputs_targets(train_tagged_preprocessed_target, 
                                                                              target_tokenizer)

Each token of a *decoder_input* sentence will be fed to the decoder as the **next** expected token, and each token of a *decoder_target* sentence will be used to calculate the loss against the decoder's actual output; exactly as we covered in the slides.

In [ ]:
print(train_decoder_inputs[0], train_decoder_targets[0])

print(target_tokenizer.sequences_to_texts(train_decoder_inputs[:1]), 
      target_tokenizer.sequences_to_texts(train_decoder_targets[:1]))

Exactly as we did in the previous [RNN demo](https://github.com/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemystified_recurrent_neural_networks.ipynb), we'll pad all sequences to a uniform length.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

In [ ]:
max_encoding_len = len(max(train_encoder_inputs, key=len))
max_encoding_len

In [ ]:
max_decoding_len = len(max(train_decoder_inputs, key=len))
max_decoding_len

In [ ]:
padded_train_encoder_inputs = pad_sequences(train_encoder_inputs, max_encoding_len, padding='post', truncating='post')
padded_train_decoder_inputs = pad_sequences(train_decoder_inputs, max_decoding_len, padding='post', truncating='post')
padded_train_decoder_targets = pad_sequences(train_decoder_targets, max_decoding_len, padding='post', truncating='post')

In [ ]:
print(padded_train_encoder_inputs[0])
print(padded_train_decoder_inputs[0])
print(padded_train_decoder_targets[0])

When converting a padded sequence back to text, padding is replaced with the out-of-vocabulary *\<unk\>* token.

In [ ]:
target_tokenizer.sequences_to_texts([padded_train_decoder_inputs[0]])

The training dataset is now ready, and we can follow the same preprocessing steps to prepare the validation dataset.

In [ ]:
# Download validation set pairs.
!wget https://raw.githubusercontent.com/nitinpunjabi/nlp-demystified/main/datasets/hun_eng_pairs/hun_eng_pairs_val.txt

In [ ]:
with open('hun_eng_pairs_val.txt') as file:
  val = [line.rstrip() for line in file]

In [ ]:
def process_dataset(dataset):

  # Split the Hungarian and English sentences into separate lists.
  input, output = map(list, zip(*[pair.split(SEPARATOR) for pair in dataset]))

  # Unicode normalization and inserting spaces around punctuation.
  preprocessed_input = [preprocess_sentence(s) for s in input]
  preprocessed_output = [preprocess_sentence(s) for s in output]

  # Tag target sentences with <sos> and <eos> tokens.
  tagged_preprocessed_output = tag_target_sentences(preprocessed_output)

  # Vectorize encoder source sentences.
  encoder_inputs = source_tokenizer.texts_to_sequences(preprocessed_input)

  # Vectorize and create decoder input and target sentences.
  decoder_inputs, decoder_targets = generate_decoder_inputs_targets(tagged_preprocessed_output, 
                                                                    target_tokenizer)
  
  # Pad all collections.
  padded_encoder_inputs = pad_sequences(encoder_inputs, max_encoding_len, padding='post', truncating='post')
  padded_decoder_inputs = pad_sequences(decoder_inputs, max_decoding_len, padding='post', truncating='post')
  padded_decoder_targets = pad_sequences(decoder_targets, max_decoding_len, padding='post', truncating='post')

  return padded_encoder_inputs, padded_decoder_inputs, padded_decoder_targets


In [ ]:
# Process validation dataset
padded_val_encoder_inputs, padded_val_decoder_inputs, padded_val_decoder_targets = process_dataset(val)

We're now ready to build our translation model. These are the parameters we'll use (feel free to try different values).

In [ ]:
embedding_dim = 128
hidden_dim = 256
default_dropout=0.2
batch_size = 32
epochs = 30

Unlike the models we've built up to this point, this model will have **two** inputs:<br>
1. The encoder receives the source sentences (Hungarian) and generates the initial state inputs for the decoder.
2. The decoder receives the decoder input sentences (English) for teacher forcing.
<br>

Since that's the case, we won't use the Keras **Sequential API**. Rather, we'll use the **Functional API** which will give us more flexibility in expressing our model layers as graphs. Fortunately, the work remains pretty intuitive as long as we keep in mind what's happening behind the scenes.
<br>

https://keras.io/api/models/sequential/<br>
https://www.tensorflow.org/guide/keras/functional/

The code in the following cell specifies the encoder. If you haven't used the **Functional API** before, the most important thing to realize here is this: Each layer is like a _node_ in a graph. Each time we pass the output of one layer to the next, it creates a link between the two _nodes_ and allows data to flow through. Once the whole training model is built, we'll visualize it for greater clarity.

In [ ]:
# The initial encoder input layer which will take in padded sequences. We're specifying
# a shape of None here but you can specify it upfront if you want since we
# know what the max encoding length is.
encoder_inputs = layers.Input(shape=[None], name='encoder_inputs')

# The embedding layer. Similar to what we did in the RNN demo.
encoder_embeddings = layers.Embedding(source_vocab_size, 
                                      embedding_dim,
                                      mask_zero=True,
                                      name='encoder_embeddings')

# Passing the input layer output to the embedding layer creates a link between the
# two. Input sequences will now flow into the embedding layer which will output
# a sequence of embeddings.
encoder_embedding_output = encoder_embeddings(encoder_inputs)


# We're not using any kind of attention mechanism in this model, so setting only
# return_state to True is enough. return_sequences remains False.
encoder_lstm = layers.LSTM(hidden_dim, 
                           return_state=True, 
                           dropout=default_dropout, 
                           name='encoder_lstm')

# Passing the embedding layer output to the LSTM layer creates another link.
# IMPORTANT: The LSTM always returns three values. When return_sequences is
# False, encoder_outputs and state_h are the SAME. When return_sequences is
# True, encoder_outputs contains the encoder hidden states from each time step.
#
# Side note: we won't be using encoder_outputs here so that variable can be 
# replaced with a _ if preferred.
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding_output)

# The final hidden and cell/context states from the encoder will be the the
# initial states for the decoder.
encoder_states = (state_h, state_c)

The code for specifying the decoder is similar except for three additions:
1. The LSTM has *return_sequences* set to *True* since we'll need the hidden state outputted at each timestep (similar to our PoS tagger and language model from the [RNN demo](https://github.com/nitinpunjabi/nlp-demystified/blob/main/notebooks/nlpdemystified_recurrent_neural_networks.ipynb)).

2. The decoder's LSTM takes an *initial_state*, the value for which is from the encoder.

3. There's a *softmax* layer in the end to generate a probability distribution over the target (English) vocabulary.

In [ ]:
decoder_inputs = layers.Input(shape=[None], name='decoder_inputs')


decoder_embeddings = layers.Embedding(target_vocab_size, 
                                      embedding_dim, 
                                      mask_zero=True,
                                      name='decoder_embeddings')


decoder_embedding_output = decoder_embeddings(decoder_inputs)

# Return sequences set to True.
decoder_lstm = layers.LSTM(hidden_dim,
                           return_sequences=True,
                           return_state=True,
                           dropout=default_dropout,
                           name='decoder_lstm')


# Set the decoder's initial state to the encoder's final output states. Since
# return_sequences is set to True, decoder_outputs is going to be a collection of
# the decoder's hidden state at each timestep. Also note that since we don't need
# the decoder's final hidden output and cell states, those are just set to _.
decoder_outputs, _, _ = decoder_lstm(decoder_embedding_output, initial_state=encoder_states)

# Have a softmax layer in the end to create a probability distribution for the output word.
decoder_dense = layers.Dense(target_vocab_size, activation='softmax', name='decoder_dense')

# The probability distribution for the output word.
y_proba = decoder_dense(decoder_outputs)


We can now create our model and specify that it has **two** inputs and one output.
<br><br>
Note that *accuracy* is a crude metric for translation models and just serves as a proxy for how our model is doing. As we covered in the slides, the popular performance metric is BLEU.

In [ ]:
# Note how the model is taking two inputs in an array.
model = tf.keras.Model([encoder_inputs, decoder_inputs], y_proba, name='hun_eng_seq2seq_nmt_no_attention')

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',  metrics='sparse_categorical_accuracy')
model.summary()

We can visualize our model to get a better idea of the flow we've built.

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='hun_eng_seq2seq_nmt_no_attention.png', show_shapes=True, show_layer_names=True)

I personally found keeping the right matrix dimensions in mind more challenging and tricky than the theoretical concepts. So here are the dimensions we're working with in this training model.

In [ ]:
print('encoder_inputs layer\n input dimension {}\n output dimension: {}'.format((batch_size, max_encoding_len), (batch_size, max_encoding_len)))
print()
print('encoder_embeddings layer\n input dimension {}\n output dimension: {}'.format((batch_size, max_encoding_len), (batch_size, max_encoding_len, embedding_dim)))
print()
print('encoder_lstm layer\n input dimension {}\n output dimension: {}'.format((batch_size, max_encoding_len, embedding_dim), [(batch_size, hidden_dim), (batch_size, hidden_dim), (batch_size, hidden_dim)]))
print()
print()
print('decoder_inputs layer\n input dimension {}\n output dimension: {}'.format((batch_size, max_decoding_len), (batch_size, max_decoding_len)))
print()
print('decoder_embeddings layer\n input dimension {}\n output dimension: {}'.format((batch_size, max_decoding_len), (batch_size, max_decoding_len, embedding_dim)))
print()
print('decoder_lstm layer\n input dimension {}\n output dimension: {}'.format([(batch_size, max_decoding_len, embedding_dim), (batch_size, hidden_dim), (batch_size, hidden_dim)], [(batch_size, max_decoding_len, hidden_dim), (batch_size, hidden_dim), (batch_size, hidden_dim)]))
print()
print('decoder_dense layer(softmax)\n input dimension {}\n output dimension: {}'.format((batch_size, max_decoding_len, hidden_dim), (batch_size, max_decoding_len, target_vocab_size)))

Because this model takes a while to train, we're using model checkpoints to save the weights after every epoch. This way, if something goes wrong with our system during training, we can reload the last set of weights from the checkpoint, and resume training from there.
https://keras.io/api/callbacks/model_checkpoint/

In [ ]:
# Saving this to a folder on my local machine.
filepath="./HunEngNMTNoAttention/training1/cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                 save_weights_only=True,
                                                 verbose=1)

We'll also use **early stopping** as we have recently.
<br><br>
The *fit* method below is commmented out because I trained the model ahead of time and saved it. I also saved the tokenizers.
<br><br>
If you want to train it yourself, feel free to uncomment and execute it. Keep in mind that because of the random weight initialization, your trained model's output will likely differ from mine.

In [ ]:
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# history = model.fit([padded_train_encoder_inputs, padded_train_decoder_inputs], padded_train_decoder_targets,
#                      batch_size=batch_size,
#                      epochs=epochs,
#                      validation_data=([padded_val_encoder_inputs, padded_val_decoder_inputs], padded_val_decoder_targets),
#                      callbacks=[cp_callback, es_callback])

The model I previously trained exited at epoch 12 with these metrics.

> Epoch 12: saving model to ./HunEngNMTNoAttention/training1/cp.ckpt
2771/2771 [==============================] - 105s 38ms/step 
- loss: 0.1315 - sparse_categorical_accuracy: 0.8450 
- val_loss: 0.3996 - val_sparse_categorical_accuracy: 0.6832



If you choose to train your own model, the following functions save the model and tokenizers.

In [ ]:
###### Save the model.
# model.save('hun_eng_s2s_nmt_no_attention')


###### Zip and download the model.
# !zip -r ./hun_eng_s2s_nmt_no_attention.zip ./hun_eng_s2s_nmt_no_attention
# files.download("./hun_eng_s2s_nmt_no_attention.zip")


###### Save the tokenizers as JSON files. The resulting files can be downloaded by left-clicking on them.
# source_tokenizer_json = source_tokenizer.to_json()
# with io.open('source_tokenizer.json', 'w', encoding='utf-8') as f:
#   f.write(json.dumps(source_tokenizer_json, ensure_ascii=False))

# target_tokenizer_json = target_tokenizer.to_json()
# with io.open('target_tokenizer.json', 'w', encoding='utf-8') as f:
#   f.write(json.dumps(target_tokenizer_json, ensure_ascii=False))

At this point, we can retrieve the saved model and saved tokenizers and try them on the **test** dataset.

In [ ]:
# Retrieve the tokenizers.
!wget https://github.com/nitinpunjabi/nlp-demystified/raw/main/models/nmt_no_attention/hun_eng_s2s_nmt_no_attention_tokenizers.zip

In [ ]:
!unzip -o hun_eng_s2s_nmt_no_attention_tokenizers.zip

The tokenizers can be loaded using the *tokenizer_from_json* method.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/tokenizer_from_json

In [ ]:
with open('source_tokenizer.json') as f:
    data = json.load(f)
    source_tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(data)

with open('target_tokenizer.json') as f:
    data = json.load(f)
    target_tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(data)

In [ ]:
# Retrieve the model.
!wget https://github.com/nitinpunjabi/nlp-demystified/raw/main/models/nmt_no_attention/hun_eng_s2s_nmt_no_attention_model.zip

In [ ]:
!unzip -o hun_eng_s2s_nmt_no_attention_model.zip

In [ ]:
# Load the model.
model = tf.keras.models.load_model('hun_eng_s2s_nmt_no_attention')

The *test* dataset contains sentences (and most certainly words) unseen by the model.

In [ ]:
# Retrieve the test dataset.
!wget https://raw.githubusercontent.com/nitinpunjabi/nlp-demystified/main/datasets/hun_eng_pairs/hun_eng_pairs_test.txt

In [ ]:
with open('hun_eng_pairs_test.txt') as file:
  test = [line.rstrip() for line in file]

In [ ]:
test[:3]

In [ ]:
# Preprocess test dataset
padded_test_encoder_inputs, padded_test_decoder_inputs, padded_test_decoder_targets = process_dataset(test)

In [ ]:
# Evaluate the model on the test set.
model.evaluate([padded_test_encoder_inputs, padded_test_decoder_inputs], padded_test_decoder_targets)

Our training model uses teacher forcing, but that won't be the case for inference. So we'll take the trained layers from our training model, and create **separate**, stand-alone encoder and decoder models. This will give us much greater control over how the output is created as we'll see. (e.g. feeding the current output as the input to the next time step).

In [ ]:
# These are the layers of our trained model.
[layer.name for layer in model.layers]

Creating a stand-alone encoder is just a matter of retrieving the trained layers by name and re-creating the graph.

The major difference here is the last line where we're calling *tf.keras.Model* to create a stand-alone encoder with *encoder_inputs* as the input and *encoder_states* as the output.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/Model#get_layer

In [ ]:
encoder_inputs = model.get_layer('encoder_inputs').input

encoder_embedding_layer = model.get_layer('encoder_embeddings')
encoder_embeddings = encoder_embedding_layer(encoder_inputs)

encoder_lstm = model.get_layer('encoder_lstm')

_, encoder_state_h, encoder_state_c = encoder_lstm(encoder_embeddings)

encoder_states = [encoder_state_h, encoder_state_c]

# Our stand-alone encoder model. encoder_inputs is the input to the encoder,
# and encoder_states is the expected output.
encoder_model_no_attention = tf.keras.Model(encoder_inputs, encoder_states)

In [ ]:
plot_model(encoder_model_no_attention, to_file='encoder_model_no_attention_plot.png', show_shapes=True, show_layer_names=True)

Creating the decoder is similar. The major difference here is that there are two additional inputs to the decoder representing its LSTM hidden state (*decoder_input_state_h*) and cell state (*decoder_input_state_c*), respectively.
<br><br>
They are there because the encoder and decoder are now separate models, so we'll manually take the encoder's output (its final states) and use them as the decoder's initial state. From there, at each time step, we'll take the decoder's state outputs and feed them to the next time step.
<br><br>
The stand-alone decoder now takes as input:
- Either the '\<sos\>' token (at the beginning) or the output from the previous timestep. Either way, it'll now take only one token at a time.
- Hidden and cell states, either from the encoder initially or from the decoder's last time step.
<br>

And it outputs a probability distribution for the current output, and new hidden and cell states.

In [ ]:
decoder_inputs = model.get_layer('decoder_inputs').input

decoder_embedding_layer = model.get_layer('decoder_embeddings')
decoder_embeddings = decoder_embedding_layer(decoder_inputs)

# Inputs to represent the decoder's LSTM hidden and cell states. We'll populate 
# these manually using the encoder's output for the initial state.
decoder_input_state_h = tf.keras.Input(shape=(hidden_dim,), name='decoder_input_state_h')
decoder_input_state_c = tf.keras.Input(shape=(hidden_dim,), name='decoder_input_state_c')
decoder_input_states = [decoder_input_state_h, decoder_input_state_c]

decoder_lstm = model.get_layer('decoder_lstm')

decoder_sequence_outputs, decoder_output_state_h, decoder_output_state_c = decoder_lstm(
    decoder_embeddings, initial_state=decoder_input_states
)

# Update hidden and cell states for the next time step.
decoder_output_states = [decoder_output_state_h, decoder_output_state_c]

decoder_dense = model.get_layer('decoder_dense')
y_proba = decoder_dense(decoder_sequence_outputs)

decoder_model_no_attention = tf.keras.Model(
    [decoder_inputs] + decoder_input_states, 
    [y_proba] + decoder_output_states
)

In [ ]:
plot_model(decoder_model_no_attention, to_file='decoder_model_no_attention_plot.png', show_shapes=True, show_layer_names=True)

The method below translates a sentence from the source language to the target language. It encodes the source sentence as usual, then feeds the encoder's state outputs and the \<sos\> token into the decoder.
<br><br>
The decoder's outputs (the resulting word and its hidden/cell states) are then fed back to the decoder at the next time step. This continues until either a max word limit is reached or an \<eos\> token is generated.

In [ ]:
def translate_without_attention(sentence: str, 
                                source_tokenizer, encoder,
                                target_tokenizer, decoder,
                                max_translated_len = 30):

  # Vectorize the source sentence and run it through the encoder.    
  input_seq = source_tokenizer.texts_to_sequences([sentence])

  # Get the tokenized sentence to see if there are any unknown tokens.
  tokenized_sentence = source_tokenizer.sequences_to_texts(input_seq)

  states = encoder.predict(input_seq)  

  current_word = '<sos>'
  decoded_sentence = []

  while len(decoded_sentence) < max_translated_len:
    
    # Set the next input word for the decoder.
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_tokenizer.word_index[current_word]
    
    # Determine the next word.
    target_y_proba, h, c = decoder.predict([target_seq] + states)
    target_token_index = np.argmax(target_y_proba[0, -1, :])
    current_word = target_tokenizer.index_word[target_token_index]

    if (current_word == '<eos>'):
      break

    decoded_sentence.append(current_word)
    states = [h, c]
  
  return tokenized_sentence[0], ' '.join(decoded_sentence)


To test it out, we'll sample a bunch of sentences from the *test* dataset and translate them.

In [ ]:
# random.seed is just here to re-create results.
random.seed(1)
sentences = random.sample(test, 15)
sentences

In [ ]:
def translate_sentences(sentences, translation_func, source_tokenizer, encoder,
                        target_tokenizer, decoder):
  translations = {'Tokenized Original': [], 'Reference': [], 'Translation': []}

  for s in sentences:
    source, target = s.split(SEPARATOR)
    source = preprocess_sentence(source)
    tokenized_sentence, translated = translation_func(source, source_tokenizer, encoder,
                                                      target_tokenizer, decoder)

    translations['Tokenized Original'].append(tokenized_sentence)
    translations['Reference'].append(target)
    translations['Translation'].append(translated)
  
  return translations

We'll load the results into a Pandas **DataFrame** for easier viewing.<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

In [ ]:
translations_no_attention = pd.DataFrame(translate_sentences(sentences, translate_without_attention,
                                                             source_tokenizer, encoder_model_no_attention,
                                                             target_tokenizer, decoder_model_no_attention))
translations_no_attention

So performance here tends to be mixed with a few translations which are off, a few which match the reference exactly, and a few interesting ones where the translation doesn't match the reference but conveys the same idea. But overall, pretty good.
<br><br>
There are several limitations we're working with:
- Hungarian is a difficult language because of its flexible structure and high expressiveness. The conjugation style leads to several English words often mapping to one word in Hungarian. Even Google Translate tends to have a high error rate with Hungarian.
- This is a small dataset and to build something robust requires much more data. It also isn't very wide and diverse.
- We're using a simple model with one LSTM for the encoder, and one LSTM for the decoder. In contrast, when Google Translate was still fully recurrence-based, it used an eight-layer LSTM for the encoder, and an eight-layer LSTM for the decoder (though in this case, I still think the data is the primary limitation).
- Translation remains a tricky and hard problem. For examples of issues that arise in translation, check out the video from Rasa in the *References* section at the end of the notebook.
<br><br>
Still, we managed to get some decent and interesting results with a simple approach. Next, let's learn how to add an attention layer.

# Recurrence-based Seq2Seq Neural Machine Translation **WITH** Luong Attention

All the preprocessing steps and tokenizers we already have can carry over here.

This time around, we'll decouple the encoder and decoder from the beginning (i.e. won't have to build separate models for training and inference). And rather than using the **Functional API**, we'll use **subclassing** instead.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/Model<br>
https://www.tensorflow.org/guide/keras/custom_layers_and_models

In the constructor, we declare our layers. The encoder flow remains much the same. The big difference with this encoder's LSTM layer is that it has *return_sequences* set to *True*. This means the encoder will now output a hidden state at each time step which will be used by the attention mechanism. And there is no masking because masks don't flow through custom layers (such as the attention mechanism), so we'll handle the masking ourselves in a custom loss function.
<br><br>
The overriden *call* method contains the operations of our model. i.e. it describes how the model should turns inputs into outputs.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/Model#call

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(Encoder, self).__init__()

        # No masking here. We'll handle it ourselves.
        self.embedding = layers.Embedding(source_vocab_size, 
                                          embedding_dim,
                                          name='encoder_embedding_layer')
        
        # return_sequences is set to True this time.
        self.lstm = layers.LSTM(hidden_dim, 
                                return_sequences=True, 
                                return_state=True,
                                name='encoder_lstm')

    def call(self, input):
        embeddings = self.embedding(input)
        
        # output_seq will hold the encoder's hidden states from each time step.
        output_seq, state_h, state_c = self.lstm(embeddings)

        return output_seq, state_h, state_c

We can get a sense of the encoder's outputs by instantiating a dummy encoder and passing it some input.

In [ ]:
test_encoder = Encoder(source_vocab_size, embedding_dim, hidden_dim)

The encoder will receive a batch of sequences.

In [ ]:
test_encoder_batch = padded_train_encoder_inputs[:3]
print(test_encoder_batch.shape)
test_encoder_batch

The encoder's overriden *call* method is invoked by *\_\_call\_\_* (dunder method), which is why we're executing the encoder instance as if it's a function.<br>
https://stackoverflow.com/questions/9663562/what-is-the-difference-between-init-and-call<br>
https://docs.python.org/3/reference/datamodel.html#object.__call__

In [ ]:
test_encoder_outputs, state_h, state_c = test_encoder(test_encoder_batch)

In [ ]:
print(test_encoder_outputs.shape)
print(state_h.shape)
print(state_c.shape)

Before we create the attention class, let's walk through a simple example of multiplicative (dot product) attention. The one we'll eventually create will be almost identical.

Suppose the following is the sequence of hidden states from the encoder after processing a single four-token sentence. Each hidden state is of dimension three.

In [ ]:
# Sample encoder LSTM output for single sequence of length 4.
encoder_out = tf.constant([[1., 2., 3.],
                           [2., 3., 4.],
                           [3., 4., 5.],
                           [4., 5. ,6.]])

In [ ]:
print('encoder_out shape: {}'.format(encoder_out.shape))
print('Number of timesteps: {}'.format(encoder_out.shape[0]))
print('Number of hidden dimensions: {}'.format(encoder_out.shape[1]))

And suppose the following is the hidden state from the decoder at a particular time step.

In [ ]:
# Sample decoder LSTM output for a single timestep.
decoder_out = tf.constant([[1., 3., 5.]])

In [ ]:
print('decoder_out shape: {}'.format(decoder_out.shape))
print('Number of timesteps: {}'.format(decoder_out.shape[0]))
print('Number of hidden dimensions: {}'.format(decoder_out.shape[1]))

In order to get the attention scores, we need to perform a dot product between the decoder hidden state and the encoder hidden states. To do that, the encoder hidden states need to be transposed.

In [ ]:
tf.transpose(encoder_out)

The *tf.matmul* function can perform the transpose and the dot product in one step to yield the attention scores.<br>
https://www.tensorflow.org/api_docs/python/tf/linalg/matmul

In [ ]:
attention_scores = tf.matmul(decoder_out, encoder_out, transpose_b=True)
print(attention_scores)

We can then apply a softmax to the attention scores to get the attention weights.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/activations/softmax

In [ ]:
attention_weights = tf.keras.activations.softmax(attention_scores, axis=-1)
print(attention_weights)

Finally, we can create the context vector using the attention weights and the encoder outputs. Once we have our context vector we can use it in our decoder. Refer to this module's video/slides for a refresher if needed.


In [ ]:
context = tf.matmul(attention_weights, encoder_out)
print(context)

The attention class below follows the same steps. The only difference is the encoder outputs go through a dense layer first.
<br><br>
For input, the attention class takes in all the encoder hidden states and the current decoder hidden state. For output, it returns the attention weights (which could be useful for other purposes) and the context vector.

In [ ]:
class LuongAttention(tf.keras.Model):
  def __init__(self, hidden_dim):
    super(LuongAttention, self).__init__()

    self.w = layers.Dense(hidden_dim, name='encoder_outputs_dense')

  def call(self, inputs):
    encoder_output_seq, decoder_output = inputs
    z = self.w(encoder_output_seq)
    attention_scores = tf.matmul(decoder_output, z, transpose_b=True)
    attention_weights = tf.keras.activations.softmax(attention_scores, axis=-1)
    context = tf.matmul(attention_weights, encoder_output_seq)

    return attention_weights, context


The decoder remains mostly the same except:
1. The embedding layer doesn't have masking.
2. There's an additional attention step in the flow.
3. The combined context and decoder output vector goes through a dense layer *w*. Refer to this module's video/slides if a refresher is needed.
4. The final dense layer has no softmax activation. Rather, we'll calculate the loss directly on the logits.
<br>

For inputs, the decoder receives:
1. the token(s) for the current step. During training with teacher forcing, that would be the next expected token(s). During inference, that would be the token(s) generated from the last time step.
2. all the encoder's hidden states.
3. whatever the decoder should take as its current hidden and cell states.
<br>

For outputs, the decoder returns the:
1. logits.
2. last LSTM hidden and cell states.
3. attention weights.

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, hidden_dim):
    super(Decoder, self).__init__()

    self.embedding_layer = layers.Embedding(vocab_size,
                                            embedding_dim,
                                            name='decoder_embedding_layer')
    
    self.lstm = layers.LSTM(hidden_dim,
                            return_sequences=True,
                            return_state=True,
                            name='decoder_lstm')

    self.attention = LuongAttention(hidden_dim)

    self.w = tf.keras.layers.Dense(hidden_dim, activation='tanh', name='attended_outputs_dense')
    
    self.dense = layers.Dense(vocab_size, name='decoder_dense')


  def call(self, inputs):
    decoder_input, encoder_output_seq, lstm_state = inputs
    embeddings = self.embedding_layer(decoder_input)

    decoder_output, state_h, state_c = self.lstm(embeddings, initial_state=lstm_state)
   
    weights, context = self.attention([encoder_output_seq, decoder_output])

    decoder_output_with_attention = self.w(tf.concat(
        [tf.squeeze(context, 1), tf.squeeze(decoder_output, 1)], -1))

    logits = self.dense(decoder_output_with_attention)

    return logits, state_h, state_c, weights

We can get a sense of the decoder's inputs and outputs for a single training time step with a test.

In [ ]:
test_decoder = Decoder(target_vocab_size, embedding_dim, hidden_dim)

Suppose this is a batch of input sequences for the decoder (the sequences used for teacher forcing)...

In [ ]:
test_decoder_batch = padded_train_decoder_inputs[:3]
print(test_decoder_batch.shape)
test_decoder_batch

...and let's say we're currently on the second timestep (index 1).These would be the next inputs to the decoder for each sequence in the batch:

In [ ]:
test_decoder_batch[:, 1]

But we need to modify this to be *three sequences of one element* which we can do with *expand_dims*.<br>
https://www.tensorflow.org/api_docs/python/tf/expand_dims


In [ ]:
next_decoder_inputs = tf.expand_dims(test_decoder_batch[:, 1], 1)
next_decoder_inputs

In [ ]:
# Initial values for state_h and state_c are from the encoder.
test_decoder_logits, state_h, state_c, test_decoder_weights = test_decoder(
    [
      next_decoder_inputs,
      test_encoder_outputs,
      [state_h, state_c]
    ])

In [ ]:
print(test_decoder_logits.shape)
print(test_decoder_weights.shape)

We're going to use a custom loss function because our sequences contain padding, and we don't want predictions on padding to contribute to the loss.
<br><br>
This custom loss function is simply a wrapper around a **sparse categorical crossentropy** loss, but with a mask of 1s and 0s. Any target element of 0 (i.e. a padding value) will get a mask value of 0, everything else will get a mask of 1, and only target values corresponding to a mask value of 1 will be used for loss calculation.
<br><br>
Refer to this page to get a sense of what the *targets* and *logits* parameters look like:<br>
https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy

In [ ]:
def loss_func(targets, logits):
  ce_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

  mask = tf.cast(tf.math.not_equal(targets, 0), tf.float32)

  return ce_loss(targets, logits, sample_weight=mask)

We'll use Tensorflow's **Dataset** to batch our data like we did in the previous model on RNNs.<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch



In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((padded_train_encoder_inputs, 
                                              padded_train_decoder_inputs, 
                                              padded_train_decoder_targets)).batch(batch_size, drop_remainder=True)

Finally, this is our training function. It'll be called for each batch of data.
<br><br>
*train_step* doesn't have to be in a class, but by putting it inside a *Model* subclass, we can leverage the *fit* method rather than writing our own training loop.
<br><br>
*train_step* is an overridden method called by the *fit* method.<br>
https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit
<br><br>
Any operation within the _GradientTape_ context is recorded, and the variables involved are watched. We can then have Tensorflow's autodiff calculate the gradients of the variables with respect to the loss, and backpropagate it through the operations involved.
https://www.tensorflow.org/api_docs/python/tf/GradientTape<br>
https://www.tensorflow.org/guide/autodiff
<br><br>
*train_step* is wrapped in a *@tf.function* annotation for a bit of a performance boost.<br>
https://www.tensorflow.org/api_docs/python/tf/function<br>
https://www.tensorflow.org/guide/function<br>


In [ ]:
class TranslatorTrainer(tf.keras.Model):
  def __init__(self, encoder, decoder):
    super(TranslatorTrainer, self).__init__()

    self.encoder = encoder
    self.decoder = decoder

  # This method will be called by model.fit for each batch.
  @tf.function
  def train_step(self, inputs):
      loss = 0.

      encoder_input_seq, decoder_input_seq, decoder_target_seq = inputs
      
      with tf.GradientTape() as tape:
          encoder_output_seq, state_h, state_c = self.encoder(encoder_input_seq)
  
          # We need to create a loop to iterate through the target sequences
          for i in range(decoder_target_seq.shape[1]):

              # Input to the decoder must have shape of (batch_size, length)
              # so we need to expand one dimension (just like in the previous example).
              next_decoder_input = tf.expand_dims(decoder_input_seq[:, i], 1)
              logits, state_h, state_c, _ = self.decoder(
                  [next_decoder_input, encoder_output_seq, (state_h, state_c)])

              # The loss is now accumulated through the whole batch
              loss += self.loss(decoder_target_seq[:, i], logits)

      # Update the parameters and the optimizer
      variables = encoder.trainable_variables + decoder.trainable_variables
      gradients = tape.gradient(loss, variables)
      self.optimizer.apply_gradients(zip(gradients, variables))

      return {'loss': loss / decoder_target_seq.shape[1]}

In [ ]:
encoder = Encoder(source_vocab_size, embedding_dim, hidden_dim)
decoder = Decoder(target_vocab_size, embedding_dim, hidden_dim)
optimizer = tf.keras.optimizers.Adam()

translator_trainer = TranslatorTrainer(encoder, decoder)
translator_trainer.compile(optimizer=optimizer, loss=loss_func)

We now have everything to train our encoder and decoder.
<br><br>
The following *fit* call is commented out because the encoder and decoder were trained prior to save time. If you want to train them yourself, feel free to uncomment and execute the method.<br><br>
**Note**: Because of the random weight initialization, your trained model's output will likely differ from mine.

In [ ]:
epochs = 12
# translator_trainer.fit(dataset, epochs=epochs)

Functions to save, zip, and download the weights of the trained encoder and decoder.

In [ ]:
# encoder.save_weights('attention_encoder_weights_with_dropout_ckpt')
# decoder.save_weights('attention_decoder_weights_with_dropout_ckpt')

# !zip -r ./attention_weights.zip ./attention_weights

# files.download('./attention_weights.zip')

We'll download and load the previously trained attention-based encoder and decoder.

In [ ]:
!wget https://github.com/nitinpunjabi/nlp-demystified/raw/main/models/nmt_with_attention/attention_weights.zip
!unzip -o attention_weights.zip

In [ ]:
encoder.load_weights('attention_weights/attention_encoder_weights_ckpt')
decoder.load_weights('attention_weights/attention_decoder_weights_ckpt')

The *translate_with_attention* method is similar to the method without attention, except it involves padding since the LSTMs didn't use masking, and also using the encoder's hidden states.

In [ ]:
def translate_with_attention(sentence: str, 
                             source_tokenizer, encoder,
                             target_tokenizer, decoder,
                             max_translated_len = 30):
    input_seq = source_tokenizer.texts_to_sequences([sentence])
    tokenized = source_tokenizer.sequences_to_texts(input_seq)

    input_seq = pad_sequences(input_seq, maxlen=max_encoding_len, padding='post')
    encoder_output, state_h, state_c  = encoder.predict(input_seq)

    current_word = '<sos>'
    decoded_sentence = []

    while len(decoded_sentence) < max_translated_len:
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = target_tokenizer.word_index[current_word]

        logits, state_h, state_c, _ = decoder.predict([target_seq, encoder_output, (state_h, state_c)])
        current_token_index = np.argmax(logits[0])

        current_word = target_tokenizer.index_word[current_token_index]

        if (current_word == '<eos>'):
          break

        decoded_sentence.append(current_word)

    return tokenized[0], ' '.join(decoded_sentence)

We'll translate the same randomly sampled sentences from before and compare the results against the translations without attention.

In [ ]:
shorter_translations_w_attention = pd.DataFrame(translate_sentences(sentences, translate_with_attention,
                                                                    source_tokenizer, encoder,
                                                                    target_tokenizer, decoder))

In [ ]:
shorter_translations_w_attention.rename(columns={'Translation': 'Translation W/ Attention'}, inplace=True)
shorter_translations_w_attention['Translation W/O Attention'] = translations_no_attention['Translation']
shorter_translations_w_attention

In addition, let's take a look at results of translating the longest sentences in the training set to see how good they are.

In [ ]:
pairs = train.copy()
pairs.sort(key=lambda s: len(s))
longer_sentences = pairs[-10:]
longer_sentences

In [ ]:
longer_translations_wo_attention = translate_sentences(longer_sentences, translate_without_attention, 
                                                       source_tokenizer, encoder_model_no_attention,
                                                       target_tokenizer, decoder_model_no_attention)

In [ ]:
longer_translations_with_attention = pd.DataFrame(translate_sentences(longer_sentences, translate_with_attention,
                                                                      source_tokenizer, encoder,
                                                                      target_tokenizer, decoder))
longer_translations_with_attention.rename(columns={'Translation': 'Translation W/ Attention'}, inplace=True)
longer_translations_with_attention['Translation W/O Attention'] = longer_translations_wo_attention['Translation']
longer_translations_with_attention

Ok, so clearly the encoder/decoder with attention gets further along before losing signal and it often manages to translate words later in the sentence.<br><br>
But still, there are clear limitiations. For one thing, we need *much* more data. For another, most industrial translation systems use more complex models. For example, Google's NMT system (when it was based entirely on LSTMs) used eight layers for both the encoder and decoder.<br><br>
But beyond that, we can use a more powerful model which we'll explore in the next module.

# Further Exploration
1. Download another translation collection from [Tatoeba](https://tatoeba.org/en/downloads) and try using a more complex model (e.g. with more layers) to see if you get better performance.
<br><br>
2. Seq2Seq is a general problem approach. Try turning a non-language sequence into another sequence. For example, turn simple arithmetic problems (e.g. 2 + 1 * 3) into its respective answer. Generating your own dataset for such a scenario is easy.
<br><br>
3. [BLEU](https://en.wikipedia.org/wiki/BLEU) is a popular metric for evaluating translation quality. Now that you know how BLEU works (see video/slides) and have a bunch of machine-generated translations along with their human-provided references, try calculating BLEU for the results.
<br><br>
4. Implement Beam Search.

# References

Challenges in machine tanslation from RASA:
https://www.youtube.com/watch?v=94K4LU_Pe8Y

Sequence to Sequence Learning with Neural Networks

https://arxiv.org/abs/1409.3215


Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation
https://arxiv.org/abs/1406.1078


Google's Neural Machine Translation System: Bridging the Gap between Human and Machine Translation
- 8 encoder and decoder layers
- Uses attention mechanism

 https://arxiv.org/abs/1609.08144




Neural Machine Translation by Jointly Learning to Align and Translate
 https://arxiv.org/abs/1409.0473


Effective Approaches to Attention-based Neural Machine Translation
https://arxiv.org/abs/1508.04025
